In [1]:
# Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

#  Import and read Australian Vehicle Prices.csv

Vehicleprices_df = pd.read_csv("Australian Vehicle Prices.csv")
Vehicleprices_df.head()

,Brand,Year,Model,Car/Suv,Title,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,ColourExtInt,Location,CylindersinEngine,BodyType,Doors,Seats,Price
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,2022 Ssangyong Rexton Ultimate (awd),DEMO,Automatic,"4 cyl, 2.2 L",AWD,Diesel,8.7 L / 100 km,5595,White / Black,"Caringbah, NSW",4 cyl,SUV,4 Doors,7 Seats,51990
1,MG,2022.0,MG3,Hatchback,2022 MG MG3 Auto Excite (with Navigation),USED,Automatic,"4 cyl, 1.5 L",Front,Premium,6.7 L / 100 km,16,Black / Black,"Brookvale, NSW",4 cyl,Hatchback,5 Doors,5 Seats,19990
2,BMW,2022.0,430I,Coupe,2022 BMW 430I M Sport,USED,Automatic,"4 cyl, 2 L",Rear,Premium,6.6 L / 100 km,8472,Grey / White,"Sylvania, NSW",4 cyl,Coupe,2 Doors,4 Seats,108988
3,Mercedes-Benz,2011.0,E500,Coupe,2011 Mercedes-Benz E500 Elegance,USED,Automatic,"8 cyl, 5.5 L",Rear,Premium,11 L / 100 km,136517,White / Brown,"Mount Druitt, NSW",8 cyl,Coupe,2 Doors,4 Seats,32990
4,Renault,2022.0,Arkana,SUV,2022 Renault Arkana Intens,USED,Automatic,"4 cyl, 1.3 L",Front,Unleaded,6 L / 100 km,1035,Grey / Black,"Castle Hill, NSW",4 cyl,SUV,4 Doors,5 Seats,34990


In [2]:

Vehicleprices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16734 entries, 0 to 16733
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              16733 non-null  object 
 1   Year               16733 non-null  float64
 2   Model              16733 non-null  object 
 3   Car/Suv            16706 non-null  object 
 4   Title              16733 non-null  object 
 5   UsedOrNew          16733 non-null  object 
 6   Transmission       16733 non-null  object 
 7   Engine             16733 non-null  object 
 8   DriveType          16733 non-null  object 
 9   FuelType           16733 non-null  object 
 10  FuelConsumption    16733 non-null  object 
 11  Kilometres         16733 non-null  object 
 12  ColourExtInt       16733 non-null  object 
 13  Location           16284 non-null  object 
 14  CylindersinEngine  16733 non-null  object 
 15  BodyType           16452 non-null  object 
 16  Doors              151

In [3]:
# Drop Columns 
columns_to_drop = ['Seats','Doors','ColourExtInt','DriveType','Title']
Vehicleprices_df = Vehicleprices_df.drop(columns=columns_to_drop)
Vehicleprices_df = Vehicleprices_df.loc[Vehicleprices_df['Price'] != 'POA']
Vehicleprices_df



,Brand,Year,Model,Car/Suv,UsedOrNew,Transmission,Engine,FuelType,FuelConsumption,Kilometres,Location,CylindersinEngine,BodyType,Price
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,DEMO,Automatic,"4 cyl, 2.2 L",Diesel,8.7 L / 100 km,5595,"Caringbah, NSW",4 cyl,SUV,51990
1,MG,2022.0,MG3,Hatchback,USED,Automatic,"4 cyl, 1.5 L",Premium,6.7 L / 100 km,16,"Brookvale, NSW",4 cyl,Hatchback,19990
2,BMW,2022.0,430I,Coupe,USED,Automatic,"4 cyl, 2 L",Premium,6.6 L / 100 km,8472,"Sylvania, NSW",4 cyl,Coupe,108988
3,Mercedes-Benz,2011.0,E500,Coupe,USED,Automatic,"8 cyl, 5.5 L",Premium,11 L / 100 km,136517,"Mount Druitt, NSW",8 cyl,Coupe,32990
4,Renault,2022.0,Arkana,SUV,USED,Automatic,"4 cyl, 1.3 L",Unleaded,6 L / 100 km,1035,"Castle Hill, NSW",4 cyl,SUV,34990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16729,Toyota,2014.0,Alphard,SYC Cars,USED,Automatic,-,Unleaded,-,66000,"Rocklea, QLD",-,Commercial,29500
16730,Mercedes-Benz,2012.0,S-Class,USED Dealer ad,USED,Automatic,-,Diesel,-,117432,"Port Melbourne, VIC",-,NaN,39888
16731,Suzuki,2022.0,Vitara,Bremer Suzuki - New,DEMO,Automatic,"4 cyl, 1.6 L",Unleaded,6.2 L / 100 km,5165,"Dinmore, QLD",4 cyl,SUV,35280
16732,Mercedes-Benz,2016.0,GLC250,SUV,USED,Automatic,"4 cyl, 2 L",Premium,7.2 L / 100 km,85525,"Albion, QLD",4 cyl,SUV,41888


In [4]:
Vehicleprices_df = Vehicleprices_df.dropna()

In [5]:
# Convertir la columna 'Price' a tipo numérico

#Vehicleprices_df['Price'] = pd.to_numeric(Vehicleprices_df['Price'], errors='coerce')

# Verificar el tipo de datos después de la conversión

#print(Vehicleprices_df['Price'].dtype)


In [6]:
print(Vehicleprices_df.head())

           Brand    Year   Model               Car/Suv UsedOrNew Transmission  \
0      Ssangyong  2022.0  Rexton  Sutherland Isuzu Ute      DEMO    Automatic   
1             MG  2022.0     MG3             Hatchback      USED    Automatic   
2            BMW  2022.0    430I                 Coupe      USED    Automatic   
3  Mercedes-Benz  2011.0    E500                 Coupe      USED    Automatic   
4        Renault  2022.0  Arkana                   SUV      USED    Automatic   

         Engine  FuelType FuelConsumption Kilometres           Location  \
0  4 cyl, 2.2 L    Diesel  8.7 L / 100 km       5595     Caringbah, NSW   
1  4 cyl, 1.5 L   Premium  6.7 L / 100 km         16     Brookvale, NSW   
2    4 cyl, 2 L   Premium  6.6 L / 100 km       8472      Sylvania, NSW   
3  8 cyl, 5.5 L   Premium   11 L / 100 km     136517  Mount Druitt, NSW   
4  4 cyl, 1.3 L  Unleaded    6 L / 100 km       1035   Castle Hill, NSW   

  CylindersinEngine   BodyType   Price  
0             4 cyl  

In [1]:
train_dataset = Vehicleprices_df.sample(frac=0.8,random_state=0)
test_dataset = Vehicleprices_df.drop(train_dataset.index)

NameError: name 'Vehicleprices_df' is not defined

In [ ]:
sns.pairplot(train_dataset[["Year", "Kilometres", "Price"]], diag_kind="kde")

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("Price")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('Price')
test_labels = test_dataset.pop('Price')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result